### Läs in bladet "Vaccinerade kommun och ålder" från covidvaccin-filen

In [1]:
import pandas as pd
import plotly.express as px

df_vaccine = pd.read_excel(
    "Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", "Vaccinerade kommun och ålder"
)

In [2]:
df_vaccine

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30-39,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40-49,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,25,Norrbottens län,2584,Kiruna,50-59,3079,2878,2860,2482.0,NaN,0.934719,0.928873,0.806106,NaN
2896,25,Norrbottens län,2584,Kiruna,60-69,2781,2648,2633,2434.0,NaN,0.952175,0.946782,0.875225,NaN
2897,25,Norrbottens län,2584,Kiruna,70-79,2194,2115,2108,2034.0,1784.0,0.963993,0.960802,0.927074,0.813127
2898,25,Norrbottens län,2584,Kiruna,80-89,1280,1256,1253,1220.0,1091.0,0.981250,0.978906,0.953125,0.852344


In [3]:
df_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Län                  2900 non-null   int64  
 1   Län_namn             2900 non-null   object 
 2   Kommun               2900 non-null   int64  
 3   Kommun_namn          2900 non-null   object 
 4   Ålder                2900 non-null   object 
 5   Befolkning           2900 non-null   int64  
 6   Antal minst 1 dos    2900 non-null   int64  
 7   Antal minst 2 doser  2900 non-null   int64  
 8   Antal 3 doser        2320 non-null   float64
 9   Antal 4 doser        870 non-null    float64
 10  Andel minst 1 dos    2900 non-null   float64
 11  Andel minst 2 doser  2900 non-null   float64
 12  Andel 3 doser        2320 non-null   float64
 13  Andel 4 doser        870 non-null    float64
dtypes: float64(6), int64(5), object(3)
memory usage: 317.3+ KB


#### 2 a. Hur många län finns representerade i datasetet?

In [4]:
number_of_counties = df_vaccine["Län"].nunique()
print(f"Det finns {number_of_counties} län representerade.")

Det finns 21 län representerade.


#### 2 b. Hur många kommuner finns representerade i datasetet?

In [5]:
number_of_municipalities = df_vaccine["Kommun"].nunique()
print(f"Det finns {number_of_municipalities} kommuner representerade.")

Det finns 290 kommuner representerade.


#### 2 c. Hur stor är befolkningen som är representerad i datasetet?

In [6]:
population = df_vaccine["Befolkning"].sum()
print(f"Det finns {population} personer representerade.")

Det finns 9092790 personer representerade.


#### 2 d. Beräkna hur många barn under 18 år det finns i Sverige baserat på det här datasetet. Du får leta upp statistik på hur stor totala befolkningen är i Sverige.

In [7]:
#'12-15' and '16-17' are the categories with people under 18
dataset_population_under_18 = df_vaccine[df_vaccine["Ålder"].isin(["12-15", "16-17"])][
    "Befolkning"
].sum()
print(
    f"Antal personer under 18 år, baserat på datasetet: {dataset_population_under_18}"
)

# Enligt https://www.scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/
# Så var det vid tillfället jag kollade: 10 557 822 personer i Sverige

total_population = 10_557_822
percentage_under_18 = dataset_population_under_18 / total_population
print(f"Andel personer under 18 år, i Sverige: {percentage_under_18:.2%}")

Antal personer under 18 år, baserat på datasetet: 745370
Andel personer under 18 år, i Sverige: 7.06%


#### 2 e. Rita ett diagram som visar på åldersfördelningen av Sveriges befolkning.

In [8]:
age_counts = df_vaccine.groupby("Ålder")["Befolkning"].value_counts().reset_index()

fig = px.histogram(age_counts, x="Ålder", y="Befolkning", title="Åldersdistribution")
fig.show()

#### 2 f. Rita stapeldiagram för andel med minst 1 dos per län, andel två doser per län och andel tre doser per län

In [9]:
# NOTE: Jag har tolkat "Andel två doser per län" som "Andel minst 2 doser"
# eftersom det finns en kolumn som heter "Andel minst 2 doser", men inte "Andel två doser per län"...

column_names = [
    "Andel minst 1 dos",
    "Andel minst 2 doser",
    "Andel 3 doser",
]

average_data = df_vaccine.groupby("Län_namn")[column_names].mean().reset_index()

fig = px.bar(
    average_data,
    x="Län_namn",
    y=column_names,
    barmode="group",
    title="Vaccinering per Län",
    labels=dict(value="Procent", variable="Doseringstyp", Län_namn="Län"),
)

fig.update_layout(yaxis_title="Procent vaccinerade per dos", xaxis_title="Län")
fig.update_yaxes(tickformat=".2%")

fig.show()

#### 2 f2. Tag Västra Götalands län och Stockholms län och rita diagram över andelen vaccinerade med minst 1 dos, två doser, tre doser och fyra doser.

In [10]:
regions = ["Västra Götalands län", "Stockholms län"]
df_filtered = df_vaccine.loc[df_vaccine["Län_namn"].isin(regions)]

df_grouped = (
    df_filtered.groupby("Län_namn")[
        [
            "Befolkning",
            "Antal minst 1 dos",
            "Antal minst 2 doser",
            "Antal 3 doser",
            "Antal 4 doser",
        ]
    ]
    .sum()
    .reset_index()
)

df_grouped["Andel minst 1 dos"] = (
    df_grouped["Antal minst 1 dos"] / df_grouped["Befolkning"]
)
df_grouped["Andel minst 2 doser"] = (
    df_grouped["Antal minst 2 doser"] / df_grouped["Befolkning"]
)
df_grouped["Andel 3 doser"] = df_grouped["Antal 3 doser"] / df_grouped["Befolkning"]
df_grouped["Andel 4 doser"] = df_grouped["Antal 4 doser"] / df_grouped["Befolkning"]

column_names = [
    "Andel minst 1 dos",
    "Andel minst 2 doser",
    "Andel 3 doser",
    "Andel 4 doser",
]

fig = px.histogram(
    df_grouped,
    x="Län_namn",
    y=column_names,
    barmode="group",
    title="Vaccinering per Län",
    labels=dict(variable="Doseringstyp", Län_namn="Län"),
)

fig.update_yaxes(tickformat=".2%")
fig.update_layout(yaxis_title="Procent vaccinerade per dos", xaxis_title="Län")
fig.show()